# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [3]:
env = UnityEnvironment(file_name="Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print (brain)

Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [6]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
timestep = 0
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    timestep += 1
    if timestep % 25 ==0: print("\rTimestep: {}".format(timestep), end=' ')
    if done:                                       # exit loop if episode finished
        break
    
print("\nScore: {}".format(score))

Timestep: 300 
Score: 0.0


When finished, you can close the environment.
### Note
If running this sheet, I have been having problems in closing and restarting the environment.  <br>I haven't found a work-around for this. <br>So do not close it at this stage, if you want to proceed to the training stage

In [7]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [8]:
# import required libraries for training
import torch
import time

# import network and agent
from agent import Agent, ReplayBuffer

In [9]:
# check whether we have access to a GPU, if not, use a CPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Torch is currently using: {}'.format(device))

Torch is currently using: cpu


### Testing untrained agent
Test that the agent and network can interact with the Unity environment, by using an untrained agent.<br>
Note - this agent should perform no better than random action choices, because the neural network was randomly initialised

In [10]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
state_size = len(state)
print("State size: {}".format(state_size))
action_size = brain.vector_action_space_size
print('Number of actions: {}'.format(action_size))

score = 0  # initialize the score
test_agent = Agent(state_size, action_size, seed=2)
for i in range(300):
    action = test_agent.act(state)
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]
    reward = env_info.rewards[0]
    done = env_info.local_done[0]
    score += reward
    state = next_state
    if i % 50 ==0:
        print ("action chosen on step {}: {}".format(i,action))
    if done:
        break

print("Score: {}".format(score))

State size: 37
Number of actions: 4
action chosen on step 0: 3
action chosen on step 50: 0
action chosen on step 100: 3
action chosen on step 150: 3
action chosen on step 200: 3
action chosen on step 250: 3
Score: 0.0


### Train the agent using a Deep-Q network

In [11]:
from collections import deque

In [12]:
def dqn(env, n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, min_reward=13.0,
       averaging_period=100, file_save_name = 'checkpoint.pth'):
    """Deep Q Learning
    Params
    ======
        env: Unity environment
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start(float): starting value of epsilon, for epsilon-greedy action selection
        eps_end(float): lowest value of epsilon - i.e keep some exploration
        eps_decay(float): how fast to decay the epsilon rate
        min_reward(float): lowest reward required for the agent to learn
        averaging_period (int): number of most recent episodes which have to average to the min_reward
        file_save_name (str): name of file to save the checkpoint
    """
    
    # get the default brain and environment info to display
    brain_name = env.brain_names[0]
    brain = env.brains[brain_name]
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state_size = len(env_info.vector_observations[0])
    action_size = brain.vector_action_space_size
    
    print ("Environment initialised. \nState size: {}\nAction size: {}\nBrain: {}"
           .format(state_size, action_size ,brain))
    
    # intitialize the Agent
    agent = Agent(state_size=state_size, action_size=action_size, seed=42)
    print ("Agent initialized")
    
    start = time.time()
    
    scores=[]    # list containing scores for each episode
    scores_window = deque(maxlen=averaging_period)   # the most recent scores to average over
    eps = eps_start # initialize epsilon
    for i_episode in range (1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)                 # agent chooses action via the deep network
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # get the 'done' flag
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        scores_window.append(score)  # save the most recent score
        scores.append(score)         # save the most recent score
        eps = max(eps_end, eps*eps_decay) # decrease epsilon
        print ("\rEpisode: {}\tAverage score: {:.1f}".format(i_episode, np.mean(scores_window)), end='')
        if i_episode % 50 == 0:
            elapsed = time.time() - start
            print ("\rEpisode: {}\tAverage score: {:.1f}\tTime elapsed: {:.0f}s"
                   .format(i_episode, np.mean(scores_window), elapsed))
        if np.mean(scores_window) > min_reward:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), file_save_name)
            break
    print ("\nTotal time taken: {:.0f}s".format(time.time() - start))
    return scores

In [ ]:
scores = dqn(env, n_episodes=2000, min_reward = 13.0, averaging_period=100, file_save_name='checkpoint13.pth')

Environment initialised. 
State size: 37
Action size: 4
Brain: Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
Agent initialized
Episode: 50	Average score: 0.2	Time elapsed: 59s
Episode: 100	Average score: 0.6	Time elapsed: 119s
Episode: 150	Average score: 1.8	Time elapsed: 179s
Episode: 200	Average score: 3.5	Time elapsed: 249s
Episode: 250	Average score: 5.4	Time elapsed: 326s
Episode: 300	Average score: 7.3	Time elapsed: 399s
Episode: 350	Average score: 9.1	Time elapsed: 460s
Episode: 400	Average score: 10.7	Time elapsed: 528s
Episode: 445	Average score: 11.8